In [ ]:
import os
import json
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
project_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(project_directory, "data")
dataset_directory = os.path.join(project_directory, "tf_data")
model_directory = os.path.join(project_directory, "models")
model_name = "model_v1" # Change this

img_height = 200
img_width = 200
batch_size = 32

class_names = sorted([name for name in os.listdir(dataset_directory) if os.path.isdir(os.path.join(dataset_directory, name))])

Load data

In [ ]:
model = tf.keras.models.load_model(os.path.join(model_directory, f'{model_name}.keras'))
with open(os.path.join(model_directory, f'{model_name}_history.json'), 'r') as f:
    history = json.load(f)

Create a summary of the layers and training parameters

In [ ]:
model.summary()

Plot training & validation loss values

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

Create classification report

In [ ]:
y_true = []
y_pred = []
random_seat = 123

val_ds = image_dataset_from_directory(
    dataset_directory,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

for images, labels in val_ds:
    predictions = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(predictions, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

conf_matrix = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

Plot first batch with predictions

In [ ]:
val_ds_shuffled = val_ds.shuffle(buffer_size=len(val_ds))

for images, labels in val_ds_shuffled.take(1):
    predictions = model.predict(images)
    
    plt.figure(figsize=(10, 10))
    for i in range(len(images)):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class = np.argmax(predictions[i])
        confidence = np.max(predictions[i])
        true_label = class_names[labels[i]]
        predicted_label = class_names[predicted_class]
        color = "green" if predicted_label == true_label else "red"
        plt.title(f"True: {true_label}\nPred: {predicted_label} ({confidence:.2f})", color=color)
        plt.axis("off")
    plt.show()

Creates predictions from a directory of test png images

In [ ]:
def extract_number(filename):
    try:
        return int(filename.split('.')[0][3:])
    except ValueError:
        return -1

def plot_images_from_directory(model, directory, class_names, true_label, target_size=(img_height, img_width), save_to_dir=""):
    image_files = sorted(
        [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.png') and file.startswith('obj')],
        key=extract_number
    )

    
    original_images = []
    altered_images = []
    
    num_correct_predictions = 0  # Counter for correct predictions
    
    for image_file in image_files:
        try:
            img = tf.keras.preprocessing.image.load_img(image_file, target_size=target_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            original_images.append(img_array)
            altered_img_array = alter_image(img_array)
            altered_img_array = np.expand_dims(altered_img_array, axis=0)
            altered_images.append(altered_img_array)
        except Exception as e:
            print(f"Error loading image {image_file}: {e}")
    
    if not altered_images:
        print("No valid images found in the directory.")
        return
    
    altered_images = np.vstack(altered_images)
    predictions = model.predict(altered_images)
    
    # Determine the subplot grid size
    num_images = len(altered_images)
    num_cols = 4
    num_rows = (num_images + num_cols - 1) // num_cols
    
    plt.figure(figsize=(num_cols * 3, num_rows * 3 + 2))
    plt.suptitle(f"{os.path.basename(directory)}", fontsize=16)
    
    for i in range(num_images):
        ax = plt.subplot(num_rows, num_cols, i + 1)
        plt.imshow(altered_images[i].astype("uint8"))
        predicted_class = np.argmax(predictions[i])
        confidence = np.max(predictions[i])
        predicted_label = class_names[predicted_class]
        
        # Determine title color based on correctness of prediction
        title_color = "green" if predicted_label == true_label else "red"
        
        # Update num_correct_predictions if prediction is correct
        if predicted_label == true_label:
            num_correct_predictions += 1
        
        plt.title(f"Pred: {predicted_label} ({confidence:.2f})", color=title_color)
        plt.axis("off")
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])
    
    # Calculate percentage of correct predictions
    percentage_correct = (num_correct_predictions / num_images) * 100
    
    # Add percentage of correct predictions to the title
    plt.suptitle(f"{os.path.basename(directory)} - Correct Predictions: {percentage_correct:.2f}%", fontsize=16)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.subplots_adjust(wspace=0.4, hspace=0.4)
    
    if len(save_to_dir) > 0:
        plt.savefig(os.path.join(save_to_dir, f'{os.path.basename(directory)}.png'))
    plt.show()